In [48]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


import fastf1 as ff1
from fastf1 import plotting
plotting.setup_mpl()

import datetime
import plotly.express as px
import plotly.graph_objects as go

%matplotlib inline

In [49]:
import dash
from dash import dcc
from dash import html

from dash.dependencies import Input, Output

# Parameters
#### Ganti parameter ini aja langsung run

In [50]:
YEAR = 2022
RACE = 'mexico'
SESSION = 'R'

LIMIT_OUTLIER= 1000

In [51]:
#color scheme
fer = ff1.plotting.team_color('ferrari')
rbr = ff1.plotting.team_color('RBR')
mer = ff1.plotting.team_color('mercedes')
alf = ff1.plotting.team_color('alfa romeo')
ast = ff1.plotting.team_color('aston martin')
mcl = ff1.plotting.team_color('mclaren')
alt = ff1.plotting.team_color('alphatauri')
wil = ff1.plotting.team_color('williams')
alp = ff1.plotting.team_color('alpine')
has = '#adadad' #has putih, ga keliatan, dibikin agak abu biar keliaan

In [67]:
ff1.Cache.enable_cache('../F1/cache')

In [68]:
session = ff1.get_session(2022, 'mexico', 'R')
session.load()

core           INFO 	Loading data for Mexico City Grand Prix - Race [v2.2.2]
api            INFO 	Using cached data for driver_info
api            INFO 	Using cached data for timing_data
api            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
api            INFO 	Using cached data for session_status_data
api            INFO 	Using cached data for track_status_data
api            INFO 	Using cached data for car_data
api            INFO 	Using cached data for position_data
api            INFO 	Using cached data for weather_data
core           INFO 	Finished loading data for 20 drivers: ['1', '44', '11', '63', '55', '16', '3', '31', '4', '77', '10', '23', '24', '5', '18', '47', '20', '6', '14', '22']


In [54]:
SAI = session.laps.pick_driver('SAI')
PER = session.laps.pick_driver('PER')
HAM = session.laps.pick_driver('HAM')
LEC = session.laps.pick_driver('LEC')
ALO = session.laps.pick_driver('ALO')
NOR = session.laps.pick_driver('NOR')
VER = session.laps.pick_driver('VER')
MSC = session.laps.pick_driver('MSC')
VET = session.laps.pick_driver('VET')
MAG = session.laps.pick_driver('MAG')
STR = session.laps.pick_driver('STR')
LAT = session.laps.pick_driver('LAT')
RIC = session.laps.pick_driver('RIC')
TSU = session.laps.pick_driver('TSU')
OCO = session.laps.pick_driver('OCO')
GAS = session.laps.pick_driver('GAS')
BOT = session.laps.pick_driver('BOT')
RUS = session.laps.pick_driver('RUS')
ZHO = session.laps.pick_driver('ZHO')
ALB = session.laps.pick_driver('ALB')

In [55]:
df = pd.concat([SAI, PER, HAM, LEC, ALO, NOR, VER, MSC, VET,MAG,
                STR, LAT, RIC, TSU, OCO, GAS, BOT, RUS, ZHO, ALB])

In [56]:
df.reset_index(inplace=True, drop=True)

In [57]:
#convert LapTime in timedelta64 to float in seconds
df['LapTime_seconds'] = df['LapTime'].dt.seconds*1000000 + df['LapTime'].dt.microseconds
df['LapTime_seconds'] = df['LapTime_seconds']/1000000

In [58]:
# set palette dengan dictionary berdasarkan driver dan color scheme dari ff1
my_palette ={'SAI':fer,
'PER':rbr,
'HAM':mer,
'LEC':fer,
'ALO':alp,
'NOR':mcl,
'VER':rbr,
'MSC':has,
'VET':ast,
'MAG':has,
'STR':ast,
'LAT':wil,
'RIC':mcl,
'TSU':alt,
'OCO':alp,
'GAS':alt,
'BOT':alf,
'RUS':mer,
'ZHO':alf,
'ALB':wil}

In [59]:
# buang data yg IsAccurate == False
df = df.drop(df[df.IsAccurate == False].index)

# fine tuning, buang outlier yg ga logis
df = df.drop(df[df.LapTime_seconds > LIMIT_OUTLIER].index)

In [60]:
ranks = pd.DataFrame(df.groupby("Driver")["LapTime_seconds"].mean().fillna(0).sort_values(ascending=False)[::-1])
ranks.reset_index(inplace=True)
ranks = ranks.to_dict('list')

In [61]:
df['mcolour'] = np.where(
     df['Driver'] == "VER" , rbr, 
     np.where(
        df['Driver'] == 'PER', rbr,
         np.where(
        df['Driver'] == 'MER', mer,
             np.where(
        df['Driver'] == 'RUS', mer,
                 np.where(
        df['Driver'] == 'LEC', fer,
                     np.where(
        df['Driver'] == 'SAI', fer,
                         np.where(
        df['Driver'] == 'RIC', mcl,
                             np.where(
        df['Driver'] == 'NOR', mcl,
                                 np.where(
        df['Driver'] == 'GAS', alt,
                                     np.where(
        df['Driver'] == 'TSU', alt,
                                         np.where(
        df['Driver'] == 'OCO', alp,
                                             np.where(
        df['Driver'] == 'ALO', alp,
                                                 np.where(
        df['Driver'] == 'BOT', alf,
                                                     np.where(
        df['Driver'] == 'ZHO', alf,
                                                         np.where(
        df['Driver'] == 'STR', ast,
                                                             np.where(
        df['Driver'] == 'MSC', has,
                                                                 np.where(
        df['Driver'] == 'MAG', has,
                                                                     np.where(
        df['Driver'] == 'LAT', wil,
                                                                         np.where(
        df['Driver'] == 'ALB', wil, '#000000'
     ))))))))))))))))))
)

In [62]:
box = px.box(df, x='Driver', y='LapTime_seconds', category_orders=ranks)

In [63]:
app = dash.Dash(__name__)

In [64]:
app.layout = html.Div(id = 'parent', children = [
    html.H1(id = 'H1', children = 'Pace Analysis - Mexico City GP 2022', style = {'textAlign':'center',\
                                            'marginTop':40,'marginBottom':40}),
    dcc.Graph(id = 'box_plot', figure = box)    
    ]
                     )

In [66]:
if __name__ == '__main__': 
    app.run_server()

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/



dash           INFO 	Dash is running on http://127.0.0.1:8050/



 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off
